In [2]:
import time
import numpy as np
import random
import re
from typing import List, Tuple
from itertools import product
import pandas as pd

Naive simulation of the game with lists. Red is 0, Blue is 1:

In [129]:
rcards = ['1'] * 26 
bcards = ['0'] * 26
cards = rcards+bcards # total 52 cards
total = []
ngames = 100000

for gamenum in range(ngames):
    random.seed(gamenum)
    random.shuffle(cards)
    #print(cards)
    total.append(''.join(cards))


In [140]:
rcards = ['1'] * 26 
bcards = ['0'] * 26
cards = rcards+bcards # total 52 cards
total = []


for gamenum in range(ngames):
    random.seed(gamenum)
    random.shuffle(cards)
    #print(cards)
    total.append(cards)
    cards = rcards+bcards


In [155]:
rcards = ['1'] * 26 
bcards = ['0'] * 26
cards = rcards+bcards # total 52 cards
ngames = 100000
total = np.empty((ngames, 52), dtype='<U1')
seeds = np.empty((ngames,52))

for gamenum in range(ngames):
    random.seed(gamenum)
    random.shuffle(cards)
    seeds[gamenum] = gamenum
    total[gamenum] = cards.copy()  # Use cards.copy() to keep the original list intact

In [156]:
len(total)

100000

In [153]:
total

array([['0', '1', '0', ..., '0', '0', '1'],
       ['0', '0', '1', ..., '1', '1', '1'],
       ['0', '0', '1', ..., '1', '0', '0'],
       ...,
       ['1', '1', '1', ..., '0', '1', '0'],
       ['1', '0', '1', ..., '1', '0', '1'],
       ['0', '1', '1', ..., '0', '1', '0']], dtype='<U1')

In [159]:
pd.DataFrame({'Seed':seeds,'Shuffled Deck':pd.Series(total)})

ValueError: Data must be 1-dimensional, got ndarray of shape (100000, 52) instead

### The order:
- for each type of guess (8 of them)
- for ngames of each guess
- calculate the results and probabilities

In [53]:
def penney(P1_guess, P2_guess, deck):
    player1_pot=0
    player2_pot=0
    start_pos=0 ##acts a traverser across the remaining deck
    stack=len(deck) ##helps keep track of how many cards are available to look at in front of the current card (in the case of not a match we move on to the next card)
    while stack>0:
        if stack - start_pos <= 1: ##this is in the instance of if start_pos is 2 but stack length is 2 (or if start_pos is 1 and stack length is 2)
            break ##if we have only 2 or 1 card left in the deck and our starting posiiton is 1 or 2 in the list
        if len(deck)>=3: ##if its possible to look at 3 cards
            if deck[start_pos]+deck[start_pos+1]+deck[start_pos+2] == P1_guess: ##look at the card and the next 2
                player1_pot+=(start_pos+3) 
                deck=deck[start_pos+3:] #takes the current card and 2 cards in front from the deck
                stack=len(deck) ##resetting the total amt of cards we can look at in front of the current card
                start_pos=0 ##resets the starting position to 0
                continue
            if deck[start_pos]+deck[start_pos+1]+deck[start_pos+2] == P2_guess:
                player2_pot+=(start_pos+3)
                deck=deck[start_pos+3:]
                stack=len(deck)
                start_pos=0
                continue
            stack-=1 ##reduces the amt of total cards we can look at 
            start_pos+=1 ##moves on to the next card as the first card in the 3 card combo
    if player1_pot>player2_pot:
        return ('P1 wins')
    elif player2_pot>player1_pot:
        return('P2 wins')
    else:
        return ('draw')

In [54]:
penney('BBB','RBB',total[4])

KeyboardInterrupt: 

In [63]:
re.findall('011',total[4])

['011', '011', '011', '011', '011']

In [89]:
bool(1<2)

True

In [161]:
# here, p1 win is 1 and p2 win is 0
# Also, B = 1 and R = 0
outcomes = []

for sequence in total:
    stack = []
    p1score = 0
    p2score = 0
    #string = ''.join(sequence)
    for card in sequence:
        stack.append(card)
        #if len(stack) >=3:
        curstack = ''.join(stack[-3:])
        if curstack == '111':
            p1score += 1 # using "trick" method to count
            #p1score += len(stack)
            #stack = stack[:-3]
            stack = []
        elif curstack == '110':
            p2score += 1
            #p2score += len(stack)
            #stack = stack[:-3]
            stack = []
    outcome = '1' if p1score > p2score else '0' if p2score > p1score else 'T'
    outcomes.append(outcome)
    # print(p1score)
    # print(p2score)
prob_p2 = outcomes.count('0')/len(outcomes) 
prob_p1 = outcomes.count('1')/len(outcomes)
prob_tie = 1 - prob_p2 - prob_p1

In [136]:
def run_game(sequence:List[str], p1_seq:str, p2_seq:str, type:str) -> Tuple[int,int]: 
    if p1_seq == p2_seq:
        return 0, 0, 'T'
    stack = ''
    p1score = 0
    p2score = 0
    for card in sequence:
        stack+=card
        curstack = stack[-3:]
        if curstack == p1_seq:
            p1score = p1score+1  if type=='trick' else p1score+len(stack)# using "trick" method to count
            stack = ''
        elif curstack == p2_seq:
            p2score = p2score+1 if type=='trick' else p2score+len(stack)
            stack = ''
    outcome = 'p1' if p1score > p2score else 'p2' if p2score > p1score else 'T'
    
    return p1score, p2score, outcome # (if positive, p1 won. If negative, p2 won. If 0, tie)

In [38]:
print(total[1], total[0])

['1', '0', '0', '1', '1', '0', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '1', '1', '0', '1', '1', '0', '0', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1'] ['1', '0', '0', '1', '1', '0', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '1', '1', '0', '1', '1', '0', '0', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1']


In [ ]:
game_df = pd.DataFrame()

In [127]:
patterns = [
    '111',
    '110',
    '101',
    '100',
    '011',
    '010',
    '001',
    '000'
]
type = [
    'trick',
    'cards'
]

p_df = pd.DataFrame(patterns, columns=['patterns'])
p_df

,patterns
0,111
1,110
2,101
3,100
4,011
5,010
6,001
7,000


In [128]:
pattern_df = p_df.merge(p_df, how = 'cross')
pattern_df.rename(columns={'patterns_x':'p1guess','patterns_y':'p2guess'},inplace=True)
pattern_df

,p1guess,p2guess
0,111,111
1,111,110
2,111,101
3,111,100
4,111,011
...,...,...
59,000,100
60,000,011
61,000,010
62,000,001


In [129]:
decks_df = pd.Series(total, name='Decks').to_frame()
decks_df

,Decks
0,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ..."
1,"[0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, ..."
2,"[1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, ..."
3,"[1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, ..."
4,"[0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
...,...
99995,"[0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, ..."
99996,"[0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, ..."
99997,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, ..."
99998,"[1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, ..."


In [130]:
game_df = decks_df.merge(pattern_df, how = 'cross')
game_df

,Decks,p1guess,p2guess
0,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,111
1,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,110
2,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,101
3,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,100
4,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,011
...,...,...,...
6399995,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...",000,100
6399996,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...",000,011
6399997,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...",000,010
6399998,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...",000,001


In [138]:
types

NameError: name 'types' is not defined

In [20]:
for sequence in total:
    for p1_pattern in patterns:
        for p2_pattern in patterns:
            
            run_game(sequence,p1_pattern,p2_pattern,'trick')

In [67]:
patterns

['111', '110', '101', '100', '011', '010', '001', '000']

In [106]:
game_df[['Decks','p1guess','p2guess']]

,Decks,p1guess,p2guess
0,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,111
1,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,110
2,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,101
3,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,100
4,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,011
...,...,...,...
6399995,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...",000,100
6399996,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...",000,011
6399997,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...",000,010
6399998,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...",000,001


In [103]:
game_df.apply(lambda row: print(row))

0          [0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...
1          [0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...
2          [0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...
3          [0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...
4          [0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...
                                 ...                        
6399995    [1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...
6399996    [1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...
6399997    [1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...
6399998    [1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...
6399999    [1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...
Name: Decks, Length: 6400000, dtype: object
0          111
1          111
2          111
3          111
4          111
          ... 
6399995    000
6399996    000
6399997    000
6399998    000
6399999    000
Name: p1guess, Length: 6400000, dtype: object
0          111
1          110
2          101
3          100
4          011

Decks      None
p1guess    None
p2guess    None
dtype: object

In [137]:
scores_series = game_df.apply(lambda x: run_game(x.Decks,x.p1guess,x.p2guess,'trick'), axis=1)
scores_df_1 = pd.DataFrame(scores_series.to_list(), columns = ['p1tricks','p2tricks','trick_outcome'])
scores_df_1

,p1tricks,p2tricks,trick_outcome
0,0,0,T
1,1,7,p2
2,1,7,p2
3,1,6,p2
4,0,8,p2
...,...,...,...
6399995,0,7,p2
6399996,3,7,p2
6399997,3,5,p2
6399998,3,4,p2


In [134]:
scores_series = game_df.apply(lambda x: run_game(x.Decks,x.p1guess,x.p2guess,'cards'), axis=1)
scores_df_2 = pd.DataFrame(scores_series.to_list(), columns = ['p1cards','p2cards','cards_outcome'])
scores_df_2

,p1cards,p2cards,cards_outcome
0,26,0,p1
1,9,41,p2
2,3,41,p2
3,8,43,p2
4,0,49,p2
...,...,...,...
6399995,0,49,p2
6399996,9,41,p2
6399997,26,24,p1
6399998,28,22,p1


In [135]:
final_df = pd.concat([game_df,scores_df_1,scores_df_2], axis=1)
final_df

,Decks,p1guess,p2guess,p1tricks,p2tricks,trick_outcome,p1cards,p2cards,cards_outcome
0,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,111,1,0,p1,26,0,p1
1,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,110,1,7,p2,9,41,p2
2,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,101,1,7,p2,3,41,p2
3,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,100,1,6,p2,8,43,p2
4,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,011,0,8,p2,0,49,p2
...,...,...,...,...,...,...,...,...,...
6399995,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...",000,100,0,7,p2,0,49,p2
6399996,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...",000,011,3,7,p2,9,41,p2
6399997,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...",000,010,3,5,p2,26,24,p1
6399998,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...",000,001,3,4,p2,28,22,p1


In [123]:
game_df

,Decks,p1guess,p2guess
0,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,111
1,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,110
2,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,101
3,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,100
4,"[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",111,011
...,...,...,...
6395,"[1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, ...",000,100
6396,"[1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, ...",000,011
6397,"[1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, ...",000,010
6398,"[1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, ...",000,001


In [94]:
scores_df = pd.DataFrame(game_df[['Decks','p1guess','p2guess']].apply(lambda row: run_game(row.iloc[0],row.iloc[1],row.iloc[2],'trick').to_list()))
#game_df[['p1score','p2score','outcome']] = scores_df
score_df

AttributeError: 'tuple' object has no attribute 'to_list'

In [30]:
game_data = []

for p1_pattern, p2_pattern, sequence, type in product(patterns, patterns, total, type):
    p1score, p2score, outcome = run_game(sequence,p1_pattern,p2_pattern,type)
    game_data.append({
        'p1_pattern': p1_pattern,
        'p2_pattern': p2_pattern,
        'sequence': sequence,
        'type': game_type,
        'p1score': p1score,
        'p2score': p2score,
        'score_diff': score_diff
    })

In [24]:
count = 0
for p1_pattern, p2_pattern, sequence in product(patterns, patterns, total):
    count+=1
print(count)

640000


In [25]:
scores = run_game(total[0],'111','011')
print(scores)

(0, 7)


In [1]:
running = ''
running += 'A'
running

'A'

There might be a bug in this code somewhere...

Got it. The right scoring method for trick is a trick appears, take the whole deck and count it as one "trick". Your score increases by one. You don't count all the cards.

In [155]:
stack = []
p1score = 0
p2score = 0
for card in total[0]:
    stack.append(card)
    #if len(stack) >=3:
    curstack = ''.join(stack[-3:])
    print(stack)
    if curstack == '111':
        #print(curstack)
        p1score += 1 # using "trick" method to count
        #p1score += len(stack)
        #stack = stack[:-3]
        stack = []
    elif curstack == '011':
        #print(curstack)
        p2score += 1
        #p2score += len(stack)
        #tack = stack[:-3]
        stack = []
    # this is accurate to the wikipedia entry online
print(p1score)
print(p2score)

['0']
['0', '1']
['0', '1', '0']
['0', '1', '0', '0']
['0', '1', '0', '0', '0']
['0', '1', '0', '0', '0', '1']
['0', '1', '0', '0', '0', '1', '1']
['0']
['0', '1']
['0', '1', '0']
['0', '1', '0', '0']
['0', '1', '0', '0', '1']
['0', '1', '0', '0', '1', '1']
['0']
['0', '1']
['0', '1', '1']
['0']
['0', '0']
['0', '0', '0']
['0', '0', '0', '0']
['0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '1']
['0', '0', '0', '0', '0', '1', '0']
['0', '0', '0', '0', '0', '1', '0', '1']
['0', '0', '0', '0', '0', '1', '0', '1', '1']
['1']
['1', '1']
['1', '1', '0']
['1', '1', '0', '0']
['1', '1', '0', '0', '1']
['1', '1', '0', '0', '1', '1']
['0']
['0', '1']
['0', '1', '0']
['0', '1', '0', '1']
['0', '1', '0', '1', '0']
['0', '1', '0', '1', '0', '1']
['0', '1', '0', '1', '0', '1', '1']
['0']
['0', '1']
['0', '1', '0']
['0', '1', '0', '1']
['0', '1', '0', '1', '0']
['0', '1', '0', '1', '0', '1']
['0', '1', '0', '1', '0', '1', '1']
['0']
['0', '0']
['0', '0', '1']
['0', '0', '1', '1']
['0']
['0', '0']

In [162]:
prob_p2

0.801443

In [163]:
prob_p1

0.115709

In [164]:
prob_tie

0.08284799999999998

Package all of it into a function: 

In [172]:
def simulate(p1guess: str, p2guess: str, sequences: List[str]) -> Tuple[float,float,float]:  # using the "trick" method
    outcomes = []
    
    for seq in sequences:
        stack = [] # to keep track of the cards
        p1score = 0
        p2score = 0

        for card in seq:
            stack.append(card)
            #if len(stack) >=3:
            curstack = ''.join(stack[-3:])
            if curstack == p1guess:
                p1score += 1 
                stack = []
            elif curstack == p2guess:
                p2score += 1
                stack = []
        outcome = '1' if p1score > p2score else '0' if p2score > p1score else 'T'
        outcomes.append(outcome)

    prob_p2 = outcomes.count('0')/len(outcomes) 
    prob_p1 = outcomes.count('1')/len(outcomes)
    prob_tie = 1 - prob_p2 - prob_p1

    return prob_p2, prob_p1, prob_tie

In [175]:
probs = simulate(p1guess = '111', p2guess = '011', sequences = total)
print(probs[0], probs[1], probs[2])

0.994836 0.001141 0.004022999999999947


This is value approximates the probabilities in the Wikipedia entry for Penney's Game under the section "Variation with playing cards".

In [27]:
start = time.time()

rcards = ['1'] * 26 
bcards = ['0'] * 26
cards = rcards+bcards # total 52 cards
ngames = 1000000
#total = np.empty((ngames, 52), dtype='<U1')
#seeds = np.empty((ngames,52))
total = []
for gamenum in range(ngames):
    random.seed(gamenum)
    #seeds[gamenum] = gamenum
    total.append(random.sample(cards, len(cards)))  # Use cards.copy() to keep the original list intact

df = pd.DataFrame({'Decks':total})
patterns = [
    '111',
    '110',
    '101',
    '100',
    '011',
    '010',
    '001',
    '000'
]
type = [
    'trick',
    'cards'
]

p_df = pd.DataFrame(patterns, columns=['patterns'])
pattern_df = p_df.merge(p_df, how = 'cross')
pattern_df.rename(columns={'patterns_x':'p1guess','patterns_y':'p2guess'},inplace=True)
game_df = df.merge(pattern_df, how = 'cross')

scores_series = game_df.apply(lambda x: run_game(x.Decks,x.p1guess,x.p2guess), axis=1)
scores_df = pd.DataFrame(scores_series.to_list(), columns = ['p1tricks','p2tricks',
                                                             'trick_outcome','p1score_cards', 
                                                             'p2score_cards', 'cards_outcome'])

final_df = pd.concat([game_df,scores_df], axis=1)

end = time.time()
print(round((end - start),4))

final_df

2154.308


,Decks,p1guess,p2guess,p1tricks,p2tricks,trick_outcome,p1score_cards,p2score_cards,cards_outcome
0,"[1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, ...",111,111,0,0,T,0,0,T
1,"[1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, ...",111,110,1,7,p2,4,44,p2
2,"[1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, ...",111,101,1,7,p2,6,40,p2
3,"[1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, ...",111,100,1,6,p2,3,46,p2
4,"[1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, ...",111,011,0,8,p2,0,47,p2
...,...,...,...,...,...,...,...,...,...
63999995,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...",000,100,2,6,p2,6,41,p2
63999996,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...",000,011,5,4,p1,27,21,p1
63999997,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...",000,010,5,2,p1,28,20,p1
63999998,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...",000,001,5,3,p1,22,26,p2


In [36]:
start = time.time()

rcards = ['1'] * 26 
bcards = ['0'] * 26
cards = rcards+bcards # total 52 cards
ngames = 1000000
#total = np.empty((ngames, 52), dtype='<U1')
#seeds = np.empty((ngames,52))
results = []

patterns = [
    '111',
    '110',
    '101',
    '100',
    '011',
    '010',
    '001',
    '000'
]

for gamenum in range(ngames):
    random.seed(gamenum)
    #seeds[gamenum] = gamenum
    ndeck = random.sample(cards, len(cards))  # Use cards.copy() to keep the original list intact
    
    for p1guess, p2guess in product(patterns, patterns):
        result = run_game(ndeck, p1guess, p2guess)
        results.append((ndeck, p1guess, p2guess, *result))

final_df = pd.DataFrame(results, columns=['Deck', 'p1guess', 'p2guess',
                                          'p1tricks', 'p2tricks', 'trick_outcome',
                                          'p1cards', 'p2cards', 'cards_outcome'])

end = time.time()
print(round((end - start),4))

final_df

MemoryError: Unable to allocate 2.38 GiB for an array with shape (5, 64000000) and data type object

In [20]:
def run_game(sequence:List[str], p1_seq:str, p2_seq:str) -> Tuple[int,int]: 
    if p1_seq == p2_seq:
        return 0, 0, 'T', 0, 0, 'T'
    stack = ''
    p1score_trick,p1score_cards,p2score_trick,p2score_cards = 0,0,0,0

    for card in sequence:
        stack+=card
        curstack = stack[-3:]
        if curstack == p1_seq:
            p1score_trick+=1 # using "trick" method to count
            p1score_cards+=len(stack)
            stack = ''
        elif curstack == p2_seq:
            p2score_trick+=1
            p2score_cards+=len(stack)
            stack = ''

    trick_outcome = 'p1' if p1score_trick > p2score_trick else 'p2' if p2score_trick > p1score_trick else 'T'
    cards_outcome = 'p1' if p1score_cards > p2score_cards else 'p2' if p2score_cards > p1score_cards else 'T'
    
    return p1score_trick, p2score_trick, trick_outcome, p1score_cards, p2score_cards, cards_outcome # (if positive, p1 won. If negative, p2 won. If 0, tie)